## Dados Utilizados

Para a realização deste projeto, foi necessário utilizar alguns conjuntos de dados de domínio público. Nesta seção iremos entrar em mais detalhes sobre as bases utilizadas.

In [1]:
import pandas as pd
import re

### Gun Violence

DESCRIÇÃO

Fonte: [Gun Violence Data - James Ko](https://www.kaggle.com/jameslko/gun-violence-data)

In [2]:
# Load gun violence dataset
gun_violence = pd.read_csv('../databases/gun_violence.zip', compression='zip')

### State Population Totals and Components of Change: 2010-2018

DESCRIÇÃO

Fonte: [United States Census Bureau](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html)

In [3]:
# Load population dataset
population = pd.read_csv('../databases/nst-est2018-alldata.zip', compression='zip')

## Limpeza dos Dados

As bases de dados selecionadas possuem alguns dados nos quais não estamos interessados e que podemos remover para melhorar o desempenho e facilitar o trabalho.
Além disso algumas estão faltando informações ou possuem uma formatação não muito adequada

### Gun Violence

In [4]:
# Drop unnecessary columns
gun_violence = gun_violence[[
    'incident_id',
    'date',
    'state',
    'latitude',
    'longitude',
    'n_killed',
    'n_injured',
    'gun_stolen',
    'gun_type',
    'n_guns_involved',
    'participant_age',
    'participant_age_group',
    'participant_gender',
    'participant_type'
]]

In [5]:
# Drop missing values
gun_violence.dropna(inplace=True)

In [6]:
# Change date formatting
gun_violence['year'] = gun_violence['date'].map(lambda x: x[0:4])
gun_violence['month'] = gun_violence['date'].map(lambda x: x[5:7])
gun_violence['day'] = gun_violence['date'].map(lambda x: x[8:10])
gun_violence.drop(columns='date', inplace=True)

In [7]:
# Change formatting on gun_stolen, gun_type, participant_age, etc.
gun_violence['gun_stolen'] = gun_violence['gun_stolen'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['gun_type'] = gun_violence['gun_type'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['participant_age'] = gun_violence['participant_age'].map(lambda x: re.findall(r'[0-9]+', x)[1::2])
gun_violence['participant_age_group'] = gun_violence['participant_age_group'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['participant_gender'] = gun_violence['participant_gender'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['participant_type'] = gun_violence['participant_type'].map(lambda x: re.findall(r'[a-zA-Z]+', x))

### Population

In [9]:
# Drop unnecessary columns
population = population[[
    'NAME',
    'POPESTIMATE2013',
    'POPESTIMATE2014',
    'POPESTIMATE2015',
    'POPESTIMATE2016',
    'POPESTIMATE2017',
    'POPESTIMATE2018'
]]

# Rename columns
population.columns = ['state', '2013', '2014', '2015', '2016', '2017', '2018']

print(population.shape)

(57, 7)


### Participantes

Vamos criar um novo data frame com informações dos participantes dos incidentes para podermos trabalhar melhor.

In [8]:
# Create new Data Frame for participants
participants = []
for index, row in gun_violence.iterrows():
    for age, group, gender, p_type in zip(row['participant_age'], row['participant_age_group'], row['participant_gender'], row['participant_type']):
        participants.append([row['incident_id'], age, group, gender, p_type])

participants = pd.DataFrame(participants, columns=['incident_id', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_type'])

## Salvar dados limpos

In [10]:
# Save clean datasets
gun_violence.to_csv('../databases/gun_violence_clean.csv')
participants.to_csv('../databases/participants_clean.csv')
population.to_csv('../databases/population_clean.csv')